## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [7]:
# !pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리입니다. !apt-get install -y fonts-nanum

In [8]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

#import eli5
#from eli5.sklearn import PermutationImportance

## 2. Data Load

#### 2.1. 데이터 로드

In [14]:
import pandas as pd
import lightgbm

train_path = 'train.csv'
test_path  = 'test.csv'

df_train, df_test = pd.read_csv(train_path), pd.read_csv(test_path)

df_train = df_train.assign(year=df_train['계약년월'].astype('str').str[:4].astype(int)).query('year > 2022')

df_train, df_test = map(lambda df: df.assign(**{'전용면적(㎡)': df['전용면적(㎡)'] // 5}), [df_train, df_test])

df_train['price'] = df_train.groupby(['아파트명', '전용면적(㎡)'])['target'].transform('mean')

df_train = df_train.drop_duplicates(subset=['아파트명', '전용면적(㎡)'])

x, y, test = df_train[['전용면적(㎡)', '좌표X', '좌표Y']], df_train[['price']], df_test[['전용면적(㎡)', '좌표X', '좌표Y']]

pred = lightgbm.LGBMRegressor().fit(x, y).predict(test)

pd.DataFrame(pred.astype(int), columns=["target"]).to_csv('sub.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000319 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 551
[LightGBM] [Info] Number of data points in the train set: 4898, number of used features: 3
[LightGBM] [Info] Start training from score 106844.797559


In [10]:
df_test.head(2)      # test dataset에 대한 inference를 진행해보겠습니다.

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,15.0,202307,26,5,1987,...,직영,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,21.0,202308,15,10,2021,...,위탁,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0


In [11]:
# 저장된 모델을 불러옵니다.
#with open('saved_model.pkl', 'rb') as f:
#    model = pickle.load(f)

In [12]:
%%time
X_test = df_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = model.predict(X_test)

KeyError: "['target'] not found in axis"

In [ ]:
real_test_pred          # 예측값들이 출력됨을 확인할 수 있습니다.

array([219400., 239300., 267100., ...,  76400.,  68540.,  69500.])

## 7. Output File Save

In [ ]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)